In [ ]:
import os
import subprocess
import math
from PIL import Image
input_dir = "../video"
output_base_dir = "../keyframe_information/keyframe/"
def reduce_frame(folder, new_folder, reduce_ratio=0.5):
    for frame_path in os.listdir(folder):
        full_frame_path = os.path.join(folder, frame_path)

        if os.path.isfile(full_frame_path) and frame_path.endswith('.jpg'):
            frame_name = frame_path.replace('.jpg', '.webp')
            image = Image.open(full_frame_path)
            x, y = image.size
            x_new, y_new = math.floor(x * reduce_ratio), math.floor(y * reduce_ratio)
            image = image.resize((x_new, y_new))
            image.save(os.path.join(new_folder, frame_name), optimize=True, quality=30, format="webp")
            print(f"Saved reduced frame: {new_folder}/{frame_name}")

for idx, input_video in enumerate(os.listdir(input_dir), start=0):
    if input_video == ".DS_Store":
        continue
    
    input_video_path = os.path.join(input_dir, input_video)
    video_name = os.path.splitext(input_video)[0]
    parent_folder_name = f"Video{idx}"
    parent_folder_path = os.path.join(output_base_dir, parent_folder_name)
    keyframe_dir = os.path.join(parent_folder_path, video_name)
    reduced_keyframe_dir = os.path.join(parent_folder_path, f"{video_name}_reduced")
    
    if not os.path.exists(keyframe_dir):
        os.makedirs(keyframe_dir)
    if not os.path.exists(reduced_keyframe_dir):
        os.makedirs(reduced_keyframe_dir)

    ffprobe_command = [
        "../packages/ffprobe", "-loglevel", "error", 
        "-select_streams", "v:0", "-show_entries", "packet=pts_time,flags", 
        "-of", "csv=print_section=0", input_video_path
    ]
    ffprobe_result = subprocess.run(ffprobe_command, stdout=subprocess.PIPE, text=True)

    keyframe_lines = [line.split(",")[0] for line in ffprobe_result.stdout.splitlines() if ",K" in line]
    for index, timestamp in enumerate(keyframe_lines, start=1):
        output_image = os.path.join(keyframe_dir, f"keyframe_{index}_{timestamp}.jpg")
        
        ffmpeg_command = [
            "ffmpeg", "-loglevel", "error", 
            "-ss", timestamp, "-i", input_video_path, 
            "-frames:v", "1", output_image
        ]
        
        subprocess.run(ffmpeg_command)
        print(f"Keyframe extracted: {output_image}")
    reduce_frame(keyframe_dir, reduced_keyframe_dir)

    print(f"Keyframes and reduced keyframes saved for {input_video} in {parent_folder_name}")
